In [1]:
# pip install dash folium dash-html-components

In [2]:
import requests
import folium
import time
import numpy as np
from dash import Dash, html, dcc
from dash.dependencies import Input, Output

In [3]:
API_KEY = "N8xhZrbfY7NZEqbqIeEedz4xUBmXARBG"

# Visualization from fetched data

In [12]:
def get_traffic_flow(api_key, location, 
                     baseURL = "api.tomtom.com", 
                     versionNumber = 4, 
                     style = "absolute", 
                     zoom = 22, 
                     form = "json"):
    """
    fetch real-time data on road speeds and travel times from TomTom Traffic API.
    """
    url = f"https://{baseURL}/traffic/services/{versionNumber}/flowSegmentData/{style}/{zoom}/{form}?key={api_key}&point={location}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Failed to fetch data. Status code: {response.status_code}"}

def get_color_by_speed(speed):
    """
    returns color based on speed:
    - Gray: speed <= 1 km/h
    - Red: 1 < speed <= 60 km/h
    - Yellow: 60 < speed <= 120 km/h
    - Green: speed > 120 km/h
    """
    if speed <= 1:
        return "gray"
    elif 1 < speed <= 60:
        return "red"
    elif 60 < speed <= 120:
        return "yellow"
    else:
        return "green"

def generate_traffic_map(api_key):
    # Define Manhattan boundaries (approximate)
    nyc_lat_min, nyc_lat_max = 40.679548, 40.882214
    nyc_lon_min, nyc_lon_max = -74.010773, -73.906158

    # Generate a grid of points across Manhattan
    lat_intervals = np.arange(nyc_lat_min, nyc_lat_max, 0.005)  # Smaller interval for more detail: [0.01, 0.005]
    lon_intervals = np.arange(nyc_lon_min, nyc_lon_max, 0.005)

    # Initialize map for visualization
    nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=22) # zoom_start = 12 (0..22)

    # Loop through the grid and fetch traffic flow data for each location
    cnt = 0
    for lat in lat_intervals:
        for lon in lon_intervals:
            location = f"{lat},{lon}"
            data = get_traffic_flow(api_key, location)
            cnt += 1
            
            if 'flowSegmentData' in data:
                speed = data['flowSegmentData']['currentSpeed']
                color = get_color_by_speed(speed)
                
                # Extract coordinates for the road segment from the response
                coordinates = data['flowSegmentData']['coordinates']['coordinate']
                
                # Convert coordinates into a format that folium.PolyLine can use
                coords = [(coord['latitude'], coord['longitude']) for coord in coordinates]
                
                # Add colored polyline (representing the road segment) to the map
                folium.PolyLine(
                    locations=coords,
                    color=color,
                    weight=5,
                    opacity=0.7
                ).add_to(nyc_map)
            

    print(f"{cnt} API requests made.")
    
    # Save map as HTML file
    nyc_map.save("manhattan_traffic_flow_.html")
    
    # return nyc_map._repr_html_()

### Main

In [13]:
# Call the function to generate and visualize the traffic map
generate_traffic_map(API_KEY)

861 API requests made.


**Report:**
- with 0.01: 231 API requests needed for 1 call (4 hours = 4 calls = 924 requests)
- with 0.005: **861** API requests needed for 1 call (4 hours = 4 calls = 3444 requests) : **current code**

**Depends on the level of zoom, results vary.**

In [ ]:
###### NOTE: it might be needed for later to schedule requests ######

# start_time = time.time()
# ########################################
# app = Dash(__name__)

# # Dash app layout
# app.layout = html.Div([
#     html.H1("NYC Traffic Flow Dashboard"),
#     dcc.Interval(id='interval-component', interval=60*60*1000, n_intervals=0),  # Refresh every hour
#     html.Iframe(id="traffic-map", srcDoc=generate_traffic_map(), width="100%", height="600")
# ])

# # Callback to refresh the map every hour
# @app.callback(
#     Output('traffic-map', 'srcDoc'),
#     Input('interval-component', 'n_intervals')
# )

# def update_map(n):
#     return generate_traffic_map()

# app.run_server(debug=True)
# ########################################
# end_time = time.time()
# print(f"Running time: {end_time - start_time} seconds")

# Visualization not from fetched data

In [1]:
import requests
import folium
from PIL import Image
from io import BytesIO

In [10]:
# def get_tile(z, x, y):
#     BASE_URL = 'https://api.tomtom.com/traffic/map/4/tile/flow/absolute/{z}/{x}/{y}.png?key=' + API_KEY
#     url = BASE_URL.format(z=z, x=x, y=y)
#     response = requests.get(url)
#     if response.status_code == 200:
#         image = Image.open(BytesIO(response.content))
#         return image
#     else:
#         print(f"Failed to retrieve tile ({z}, {x}, {y})")
#         return None
    
# get_tile(12, 2044, 1360)

In [2]:
def plot_traffic_map():
    nyc_lat = 40.7128
    nyc_lon = -74.0060
    map_nyc = folium.Map(location=[nyc_lat, nyc_lon], zoom_start=12)

    # Using folium to display map with traffic tiles
    # For folium, we can add an overlay that will provide context for the fetched tile
    folium.TileLayer('https://api.tomtom.com/traffic/map/4/tile/flow/relative/{z}/{x}/{y}.png?key=' + API_KEY,
                     attr='TomTom Traffic Flow', name='Traffic Flow', overlay=True).add_to(map_nyc)

    folium.LayerControl().add_to(map_nyc)
    # Save the map to an HTML file and open it in the browser
    map_nyc.save('nyc_traffic_flow.html')

In [5]:
plot_traffic_map()